In [1]:
import requests
import pandas as pd
import numpy as np

from FPL import FPL

import warnings
warnings.filterwarnings("ignore")

# Setting up

In [2]:
url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
response = requests.get(url)
json = response.json()
df = FPL(json).getPlayerDf()

stat = 'form'

# Analyzing

In [3]:
# Sorting by
df.sort_values(stat, ascending=False).head(10)


,second_name,team,element_type,now_cost,minutes,value_season,total_points,form,position
208,Havertz,Chelsea,3,79,1300.0,10.8,85,8.8,Midfielder
56,Cash,Aston Villa,2,53,2567.0,22.6,120,8.5,Defender
373,Salah,Liverpool,3,133,2259.0,16.7,222,8.2,Midfielder
370,Mané,Liverpool,3,117,2226.0,12.0,140,8.0,Midfielder
566,Kane,Spurs,4,125,2331.0,9.8,123,7.6,Forward
374,Robertson,Liverpool,2,72,1934.0,19.7,142,7.4,Defender
371,Ramses Becker,Liverpool,1,60,2430.0,23.0,138,7.0,Goalkeeper
369,van Dijk,Liverpool,2,68,2430.0,22.1,150,6.8,Defender
364,Matip,Liverpool,2,49,2070.0,23.1,113,6.6,Defender
451,Sancho,Man Utd,3,89,1489.0,9.2,82,6.5,Midfielder


In [4]:
# Avg value per position
position_pivot = FPL(json).createPivot('position', stat)

position_pivot.head()

,position,form
2,Goalkeeper,2.812000
0,Defender,2.326230
3,Midfielder,2.146154
1,Forward,2.034000


In [5]:
# Avg per team
team_pivot = FPL(json).createPivot('team', stat)

team_pivot.head()

,team,form
11,Man City,3.313333
5,Chelsea,3.310000
10,Liverpool,3.309091
1,Aston Villa,2.877778
16,Spurs,2.755556


In [6]:
# Goalkeepers
goal_df = FPL(json).dfPerPosition('Goalkeeper', stat)

goal_df.head()

,second_name,team,element_type,now_cost,minutes,value_season,total_points,form,position
371,Ramses Becker,Liverpool,1,60,2430.0,23.0,138,7.0,Goalkeeper
210,Mendy,Chelsea,1,61,2160.0,16.7,102,4.8,Goalkeeper
45,Martínez,Aston Villa,1,55,2520.0,18.7,103,4.0,Goalkeeper
293,Schmeichel,Leicester,1,48,2340.0,18.1,87,3.8,Goalkeeper
221,Guaita,Crystal Palace,1,46,2115.0,19.3,89,3.6,Goalkeeper


In [7]:
# Defenders
def_df = FPL(json).dfPerPosition('Defender', stat)

def_df.head(10)

,second_name,team,element_type,now_cost,minutes,value_season,total_points,form,position
56,Cash,Aston Villa,2,53,2567.0,22.6,120,8.5,Defender
374,Robertson,Liverpool,2,72,1934.0,19.7,142,7.4,Defender
369,van Dijk,Liverpool,2,68,2430.0,22.1,150,6.8,Defender
364,Matip,Liverpool,2,49,2070.0,23.1,113,6.6,Defender
212,Chalobah,Chelsea,2,47,1177.0,17.7,83,6.5,Defender
377,Alexander-Arnold,Liverpool,2,85,2313.0,22.0,187,6.2,Defender
569,Doherty,Spurs,2,48,671.0,10.4,50,5.7,Defender
406,Cancelo,Man City,2,69,2417.0,23.5,162,5.2,Defender
479,Schär,Newcastle,2,43,1690.0,15.1,65,5.0,Defender
52,Mings,Aston Villa,2,50,2378.0,19.8,99,4.8,Defender


In [8]:
# Midfielders
mid_df = FPL(json).dfPerPosition('Midfielder', stat)

mid_df.head(10)

,second_name,team,element_type,now_cost,minutes,value_season,total_points,form,position
208,Havertz,Chelsea,3,79,1300.0,10.8,85,8.8,Midfielder
373,Salah,Liverpool,3,133,2259.0,16.7,222,8.2,Midfielder
370,Mané,Liverpool,3,117,2226.0,12.0,140,8.0,Midfielder
401,De Bruyne,Man City,3,117,1531.0,10.5,123,6.5,Midfielder
451,Sancho,Man Utd,3,89,1489.0,9.2,82,6.5,Midfielder
19,Saka,Arsenal,3,67,2135.0,20.6,138,6.5,Midfielder
593,Kulusevski,Spurs,3,63,590.0,7.1,45,6.0,Midfielder
229,Zaha,Crystal Palace,3,68,2021.0,14.7,100,6.0,Midfielder
404,Mahrez,Man City,3,87,1203.0,13.1,114,6.0,Midfielder
77,Coutinho Correia,Aston Villa,3,75,773.0,7.9,59,5.3,Midfielder


In [9]:
# Forwards
fwd_df = FPL(json).dfPerPosition('Forward', stat)

fwd_df.head(10)

,second_name,team,element_type,now_cost,minutes,value_season,total_points,form,position
566,Kane,Spurs,4,125,2331.0,9.8,123,7.6,Forward
84,Toney,Brentford,4,67,2098.0,16.0,107,6.0,Forward
457,dos Santos Aveiro,Man Utd,4,122,1914.0,9.6,117,5.0,Forward
5,Lacazette,Arsenal,4,84,1523.0,9.6,81,4.7,Forward
499,Pukki,Norwich,4,58,2445.0,17.9,104,4.6,Forward
66,Ings,Aston Villa,4,77,1520.0,10.8,83,4.5,Forward
631,Hernández Suárez,Watford,4,50,1005.0,11.8,59,4.2,Forward
234,Mateta,Crystal Palace,4,52,647.0,7.3,38,4.0,Forward
542,Adams,Southampton,4,69,1645.0,13.3,92,4.0,Forward
185,Weghorst,Burnley,4,64,752.0,4.7,30,3.5,Forward
